In [4]:
################################################################################
#                              INSTALLATION GUIDE
################################################################################
# Please make sure to install these packages before running the script:
# pip install pandas 
# pip install numpy
# pip install time
# pip install requests
# pip install ete3
# pip install StringIO
# pip install re
################################################################################
#                              IMPORTS
################################################################################
import pandas as pd
import re
import numpy as np
import requests
from io import StringIO
import time
from ete3 import NCBITaxa

################################################################################
#                      GET BACTERIAL SPECIES FROM USER
################################################################################
bacterial_species_input = input("Please enter the bacterial species name (e.g., Bacteroides uniformis or Bacteroides_uniformis): ").strip()

################################################################################
#                          LOAD ORIGINAL SNIPPY DATA
################################################################################
print("\n📂 Loading input Excel file...")
file_path = 'D:/Research/snippy_final/Bacteroides_uniformis/For_machine_learning/SNPs_prioritization/input.xlsx'
df = pd.read_excel(file_path)
print("✅ Data loaded successfully!\n")

################################################################################
#              ADD BACTERIAL SPECIES COLUMN IF MISSING
################################################################################
print("🧬 Adding bacterial species column...")

if 'Bacterial_Species' not in df.columns:
    df['Bacterial_Species'] = bacterial_species_input
else:
    df['Bacterial_Species'] = df['Bacterial_Species'].fillna(bacterial_species_input)

# Clean up formatting
df['Bacterial_Species'] = df['Bacterial_Species'].str.replace('_', ' ')

# Snapshot
print("\n✅ Bacterial_Species column ready:")
print(df['Bacterial_Species'].head(3))


################################################################################
#                   STEP 2: Fetch TAX_IDs for Species
################################################################################
print("\nSTEP 2: Fetching TAX_IDs for bacterial species...")

ncbi = NCBITaxa()

def get_taxid(species_name):
    try:
        name2taxid = ncbi.get_name_translator([species_name])
        return name2taxid[species_name][0]
    except:
        return None

df['TAX_ID'] = df['Bacterial_Species'].apply(get_taxid)

# Snapshot
print("\n🔎 Sample of species with TAX_IDs:")
print(df[['Bacterial_Species', 'TAX_ID']].dropna().head(3))

################################################################################
#                   STEP 3: Extract Max Depth from EVIDENCE
################################################################################
print("\nSTEP 3: Extracting maximum depth from 'EVIDENCE' column...")

def extract_depth(evidence):
    matches = re.findall(r'\b([AGCT]):(\d+)', str(evidence))
    depths = [int(depth) for base, depth in matches]
    return max(depths) if depths else 0

df['Max_Depth'] = df['EVIDENCE'].apply(extract_depth)

# Snapshot
print("\n🔎 Sample Max Depth values:")
print(df[['EVIDENCE', 'Max_Depth']].head(3))

################################################################################
#                   STEP 4: Normalize Depth
################################################################################
print("\nSTEP 4: Normalizing Depth values...")

df['Depth'] = df['Max_Depth']
min_depth = df['Depth'].min()
max_depth = df['Depth'].max()
df['Normalized_Depth'] = (df['Depth'] - min_depth) / (max_depth - min_depth)

# Snapshot
print("\n🔎 Normalized Depth values:")
print(df[['Depth', 'Normalized_Depth']].head(3))

################################################################################
#                   STEP 5: Extract Amino Acid (AA) Changes
################################################################################
print("\nSTEP 5: Extracting amino acid changes from 'EFFECT' column...")

def extract_aa_change(effect_str):
    match = re.search(r'p\.([A-Z][a-z]{2}\d+[A-Z][a-z]{2})', str(effect_str))
    return match.group(1) if match else None

df['AA_Change'] = df['EFFECT'].apply(extract_aa_change)

# Snapshot
print("\n🔎 Sample AA Changes:")
print(df[['EFFECT', 'AA_Change']].head(3))

################################################################################
#                   STEP 6: Amino Acid Properties
################################################################################
print("\nSTEP 6: Preparing Amino Acid property dictionary...")

aa_props = {
    'Ala': {'weight': 89.1, 'polarity': 'nonpolar', 'charge': 'neutral', 'hydrophobicity': 1.8},
    'Arg': {'weight': 174.2, 'polarity': 'polar', 'charge': 'positive', 'hydrophobicity': -4.5},
    'Asn': {'weight': 132.1, 'polarity': 'polar', 'charge': 'neutral', 'hydrophobicity': -3.5},
    'Asp': {'weight': 133.1, 'polarity': 'polar', 'charge': 'negative', 'hydrophobicity': -3.5},
    'Cys': {'weight': 121.2, 'polarity': 'nonpolar', 'charge': 'neutral', 'hydrophobicity': 2.5},
    'Glu': {'weight': 147.1, 'polarity': 'polar', 'charge': 'negative', 'hydrophobicity': -3.5},
    'Gln': {'weight': 146.2, 'polarity': 'polar', 'charge': 'neutral', 'hydrophobicity': -3.5},
    'Gly': {'weight': 75.1, 'polarity': 'nonpolar', 'charge': 'neutral', 'hydrophobicity': -0.4},
    'His': {'weight': 155.2, 'polarity': 'polar', 'charge': 'positive', 'hydrophobicity': -3.2},
    'Ile': {'weight': 131.2, 'polarity': 'nonpolar', 'charge': 'neutral', 'hydrophobicity': 4.5},
    'Leu': {'weight': 131.2, 'polarity': 'nonpolar', 'charge': 'neutral', 'hydrophobicity': 3.8},
    'Lys': {'weight': 146.2, 'polarity': 'polar', 'charge': 'positive', 'hydrophobicity': -3.9},
    'Met': {'weight': 149.2, 'polarity': 'nonpolar', 'charge': 'neutral', 'hydrophobicity': 1.9},
    'Phe': {'weight': 165.2, 'polarity': 'nonpolar', 'charge': 'neutral', 'hydrophobicity': 2.8},
    'Pro': {'weight': 115.1, 'polarity': 'nonpolar', 'charge': 'neutral', 'hydrophobicity': -1.6},
    'Ser': {'weight': 105.1, 'polarity': 'polar', 'charge': 'neutral', 'hydrophobicity': -0.8},
    'Thr': {'weight': 119.1, 'polarity': 'polar', 'charge': 'neutral', 'hydrophobicity': -0.7},
    'Trp': {'weight': 204.2, 'polarity': 'nonpolar', 'charge': 'neutral', 'hydrophobicity': -0.9},
    'Tyr': {'weight': 181.2, 'polarity': 'polar', 'charge': 'neutral', 'hydrophobicity': -1.3},
    'Val': {'weight': 117.1, 'polarity': 'nonpolar', 'charge': 'neutral', 'hydrophobicity': 4.2}
}

################################################################################
#                   STEP 7: Calculate Amino Acid Impact Score
################################################################################
print("\nSTEP 7: Calculating AA Impact Scores...")

def compute_change(row):
    aa_change = row.get('AA_Change')
    if isinstance(aa_change, str) and len(aa_change) >= 6:
        ref, mut = aa_change[:3], aa_change[-3:]
        if ref in aa_props and mut in aa_props:
            weight_diff = abs(aa_props[ref]['weight'] - aa_props[mut]['weight']) / 130
            hydro_diff = abs(aa_props[ref]['hydrophobicity'] - aa_props[mut]['hydrophobicity']) / 9
            polarity_change = int(aa_props[ref]['polarity'] != aa_props[mut]['polarity'])
            charge_change = int(aa_props[ref]['charge'] != aa_props[mut]['charge'])
            return round(weight_diff + hydro_diff + polarity_change + charge_change, 3)
    return None

df['AA_Impact_Score'] = df.apply(compute_change, axis=1)

# Snapshot
print("\n🔎 Sample AA Impact Scores:")
print(df[['AA_Change', 'AA_Impact_Score']].head(3))

################################################################################
#                   STEP 8: Extract Total AA Length
################################################################################
print("\nSTEP 8: Extracting total protein lengths from 'AA_POS'...")

df['Total_AA'] = df['AA_POS'].apply(lambda x: int(str(x).split('/')[-1]) if '/' in str(x) else None)

# Snapshot
print("\n🔎 Sample Total AA Lengths:")
print(df[['AA_POS', 'Total_AA']].head(3))

################################################################################
#                   STEP 9: Extract Mutated Amino Acid Position
################################################################################

print("\nSTEP 9: Extracting mutated amino acid positions...")

df['Mutated_AA'] = df['AA_POS'].apply(lambda x: int(str(x).split('/')[0]) if '/' in str(x) else None)

# Snapshot
print("\n🔎 Sample Mutated AA Positions:")
print(df[['AA_POS', 'Mutated_AA']].head(3))


################################################################################
#                   STEP 10: Search UniProt IDs
################################################################################
print("\nSTEP 10: Searching UniProt IDs...")

def search_uniprot(gene, species_taxid, target_length, tolerance=50):
    query = f'gene:{gene} AND organism_id:{species_taxid}'
    url = f"https://rest.uniprot.org/uniprotkb/search?query={query}&fields=accession,gene_names,organism_name,length&format=tsv&size=50"
    response = requests.get(url)
    if response.status_code != 200:
        return None
    df_results = pd.read_csv(StringIO(response.text), sep='\t')
    df_results['Length_diff'] = abs(df_results['Length'] - target_length)
    close_matches = df_results[df_results['Length'].between(target_length - tolerance, target_length + tolerance)]
    if not close_matches.empty:
        return close_matches.sort_values('Length_diff').iloc[0]['Entry']
    return None

uniprot_ids = []
for index, row in df.iterrows():
    gene = str(row['GENE']).split('_')[0]
    taxid = row['TAX_ID']
    length = row['Total_AA']
    if pd.notna(gene) and pd.notna(taxid) and pd.notna(length):
        uid = search_uniprot(gene, taxid, length)
    else:
        uid = None
    uniprot_ids.append(uid)

df['UniProt_ID'] = uniprot_ids

# Snapshot
print("\n🔎 Sample UniProt IDs:")
print(df[['GENE', 'UniProt_ID']].dropna().head(3))

################################################################################
#                   STEP 11: Check Domain Presence
################################################################################
print("\nSTEP 11 (UPDATED): Checking if mutated AA lies within a domain...")

def check_domain_position(uniprot_id, mutated_position):
    if pd.isna(uniprot_id) or pd.isna(mutated_position):
        return 0
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return 0
        data = response.json()
        features = data.get('features', [])
        for feature in features:
            if feature.get('type') == 'Domain':
                begin = feature.get('location', {}).get('start', {}).get('value')
                end = feature.get('location', {}).get('end', {}).get('value')
                if begin is not None and end is not None:
                    if int(begin) <= int(mutated_position) <= int(end):
                        return 1
    except Exception:
        return 0
    return 0

df['Domain_Position_Match'] = df.apply(
    lambda row: check_domain_position(row['UniProt_ID'], row['Mutated_AA']),
    axis=1
)

# Snapshot
print("\n🔎 Sample Domain Position Match values:")
print(df[['UniProt_ID', 'Mutated_AA', 'Domain_Position_Match']].dropna().head(3))


################################################################################
#                   STEP 12: Final Priority Score
################################################################################
print("\nSTEP 12: Calculating final priority score...")

df['Final_Priority_Score'] = (2 * df['Normalized_Depth'] + df['AA_Impact_Score'] + df['Domain_Position_Match']) / 7

# Same score in percentage (0 to 100%)
df['Final_Priority_Score_Percent'] = df['Final_Priority_Score'] * 100

################################################################################
#                   STEP 13: Save Final Output
################################################################################
print("\nSaving final output file...")

final_path = "D:/Research/snippy_final/Bacteroides_uniformis/For_machine_learning/SNPs_prioritization/output.xlsx"
df.to_excel(final_path, index=False)

print("\n✅ All steps completed successfully!")
print(f"Final output saved at: {final_path}")



📂 Loading input Excel file...
✅ Data loaded successfully!

🧬 Adding bacterial species column...

✅ Bacterial_Species column ready:
0    Bacteroides uniformis
1    Bacteroides uniformis
2    Bacteroides uniformis
Name: Bacterial_Species, dtype: object

STEP 2: Fetching TAX_IDs for bacterial species...

🔎 Sample of species with TAX_IDs:
       Bacterial_Species  TAX_ID
0  Bacteroides uniformis     820
1  Bacteroides uniformis     820
2  Bacteroides uniformis     820

STEP 3: Extracting maximum depth from 'EVIDENCE' column...

🔎 Sample Max Depth values:
    EVIDENCE  Max_Depth
0  A:562 C:1        562
1  A:416 G:0        416
2  A:387 G:0        387

STEP 4: Normalizing Depth values...

🔎 Normalized Depth values:
   Depth  Normalized_Depth
0    562          1.000000
1    416          0.546584
2    387          0.456522

STEP 5: Extracting amino acid changes from 'EFFECT' column...

🔎 Sample AA Changes:
                                  EFFECT  AA_Change
0  missense_variant c.508G>T p.Val17